In [1]:
import pandas as pd
from scipy import sparse
from sklearn.metrics.pairwise import pairwise_distances

## Load `movies.csv` and `ratings.csv`
---

We'll be using the [MovieLens](https://grouplens.org/datasets/movielens/) dataset for building our recommendation engine. There are two CSVs (`movies.csv` and `ratings.csv`) that we'll eventually inner join. 

In [2]:
movies = pd.read_csv('data/movies.csv')
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [3]:
ratings = pd.read_csv('data/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


## Drop unnecessary columns

In [4]:
ratings.drop('timestamp', axis=1, inplace=True)
movies.drop('genres', axis=1, inplace=True)

## Merge `movies` and `ratings`

In [5]:
df = pd.merge(ratings, movies, on='movieId')
df.shape

(100836, 4)

In [6]:
df.head(10)

,userId,movieId,rating,title
0,1,1,4.0,Toy Story (1995)
1,5,1,4.0,Toy Story (1995)
2,7,1,4.5,Toy Story (1995)
3,15,1,2.5,Toy Story (1995)
4,17,1,4.5,Toy Story (1995)
5,18,1,3.5,Toy Story (1995)
6,19,1,4.0,Toy Story (1995)
7,21,1,3.5,Toy Story (1995)
8,27,1,3.0,Toy Story (1995)
9,31,1,5.0,Toy Story (1995)


## Create pivot table
---

1. The `title` will be the index
2. The `userId` will be the column
3. The `rating` will be the value


In [7]:
pivot = pd.pivot_table(df, index='title', columns='userId', values='rating')
pivot.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
'Hellboy': The Seeds of Creation (2004),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Round Midnight (1986),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Salem's Lot (2004),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Til There Was You (1997),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
pivot.shape

(9719, 610)

## Create sparse matrix
---

In a minute, we'll calculate the cosine similarity for each movie using the `pairwise_distances` function. Before that, we need to create a sparse matrix (datatype) using `scipy`'s `sparse` module like so:
```python
sparse.csr_matrix(pivot.fillna(0), metric='cosine')
```

In [9]:
sparse_pivot = sparse.csr_matrix(pivot.fillna(0))
print(sparse_pivot)

  (0, 609)	4.0
  (1, 331)	4.0
  (2, 331)	3.5
  (2, 376)	3.5
  (3, 344)	5.0
  (4, 112)	3.0
  (4, 344)	5.0
  (5, 20)	1.5
  (6, 11)	5.0
  (6, 18)	2.0
  (6, 90)	2.0
  (6, 94)	3.0
  (6, 171)	4.0
  (6, 216)	4.0
  (6, 287)	3.0
  (6, 293)	1.0
  (6, 306)	3.5
  (6, 376)	3.5
  (6, 413)	3.0
  (6, 473)	1.0
  (6, 476)	3.5
  (6, 519)	4.0
  (6, 554)	5.0
  (6, 560)	4.5
  (6, 598)	2.0
  :	:
  (9717, 26)	5.0
  (9717, 41)	5.0
  (9717, 56)	2.0
  (9717, 67)	4.0
  (9717, 87)	3.5
  (9717, 140)	3.5
  (9717, 197)	2.0
  (9717, 214)	2.5
  (9717, 216)	2.0
  (9717, 220)	3.5
  (9717, 238)	3.0
  (9717, 281)	4.0
  (9717, 293)	4.0
  (9717, 306)	2.5
  (9717, 312)	1.0
  (9717, 413)	3.0
  (9717, 420)	3.0
  (9717, 447)	3.0
  (9717, 473)	3.0
  (9717, 476)	3.5
  (9717, 554)	3.0
  (9717, 560)	4.0
  (9717, 596)	3.0
  (9717, 598)	2.5
  (9718, 526)	1.0


## Calculate cosine similarity
---

`sklearn` has a built-in `pairwise_distances` function that we can use for our recommender. It will return a square matrix, comparing every movie with every other movie in the dataset.

```python
pairwise_distances(sparse_pivot, metric='cosine')
```

In [10]:
recommender = pairwise_distances(sparse_pivot, metric='cosine')

## Create distances DataFrame
---

At this point, we essentially have a recommender. We'll load it into a `pandas` DataFrame for readability. 

You'll notice that each movie has a "distance" of 0 with itself (along the diagonal).

In [11]:
recommender_df = pd.DataFrame(recommender, columns=pivot.index, index=pivot.index)
recommender_df.head()

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),0.0,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.0,0.858347,1.0,...,1.0,0.657945,0.456695,0.292893,1.0,1.0,0.860569,0.672673,1.0,1.0
'Hellboy': The Seeds of Creation (2004),1.0,0.000000,0.292893,1.000000,1.000000,1.0,1.000000,1.0,1.000000,1.0,...,1.0,1.000000,1.000000,1.000000,1.0,1.0,1.000000,1.000000,1.0,1.0
'Round Midnight (1986),1.0,0.292893,0.000000,1.000000,1.000000,1.0,0.823223,1.0,1.000000,1.0,...,1.0,1.000000,1.000000,1.000000,1.0,1.0,1.000000,1.000000,1.0,1.0
'Salem's Lot (2004),1.0,1.000000,1.000000,0.000000,0.142507,1.0,1.000000,1.0,1.000000,1.0,...,1.0,1.000000,1.000000,1.000000,1.0,1.0,1.000000,1.000000,1.0,1.0
'Til There Was You (1997),1.0,1.000000,1.000000,0.142507,0.000000,1.0,1.000000,1.0,1.000000,1.0,...,1.0,1.000000,1.000000,1.000000,1.0,1.0,1.000000,1.000000,1.0,1.0


## Evaluate recommender performance
---

Now comes the fun part! Let's check out a few movies to see if the recommender aligns with our intuition. In the cell below we'll do the following:
1. Create a search term
2. Use that to find all titles matching the search query
3. For each title, we'll list off the following:
  1. The average rating
  2. The number of ratings
  3. The ten most similar movies

In [12]:
search = 'Batman'

for title in movies.loc[movies['title'].str.contains(search), 'title']:
    print(title)
    print('Average rating', pivot.loc[title, :].mean())
    print('Number of ratings', pivot.T[title].count())
    print('')
    print('10 closest movies')
    print(recommender_df[title].sort_values()[1:11])
    print('')
    print('*******************************************************************************************')
    print('')

Batman Forever (1995)
Average rating 2.9160583941605838
Number of ratings 137

10 closest movies
title
Batman (1989)                                                0.294361
True Lies (1994)                                             0.357256
Ace Ventura: Pet Detective (1994)                            0.379568
Cliffhanger (1993)                                           0.398133
Dances with Wolves (1990)                                    0.414066
Die Hard: With a Vengeance (1995)                            0.423242
Stargate (1994)                                              0.426460
Aladdin (1992)                                               0.433616
Clear and Present Danger (1994)                              0.436542
Interview with the Vampire: The Vampire Chronicles (1994)    0.439019
Name: Batman Forever (1995), dtype: float64

*******************************************************************************************

Batman (1989)
Average rating 3.4285714285714284
Number of r